In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, BaggingClassifier 
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier 
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error

Exercice sur le Bagging et le Pasting :

Contexte : Vous travaillez avec un ensemble de données sur le vin provenant du fichier wine.csv. L'objectif est de prédire le prix du vin en utilisant des modèles de Bagging et de Pasting avec des arbres de décision comme modèles de base.

    Chargement des données :
        Chargez le jeu de données depuis le fichier wine.csv.
        Supprimez les lignes avec des valeurs manquantes dans la colonne price.

    Préparation des données :
        Séparez les variables indépendantes (X) et la variable cible (y).
        Divisez l'ensemble de données en ensembles d'entraînement et de test.

    Bagging avec Arbres de Décision :
        Utilisez un modèle de Bagging avec un arbre de décision comme modèle de base.
        Utilisez le critère de Gini pour mesurer la pureté des nœuds dans les arbres.
        Ajustez le modèle sur l'ensemble d'entraînement en utilisant 50 estimateurs.
        Évaluez la performance sur l'ensemble de test en utilisant les métriques suivantes :
            Score F1
            Score R2
        Affichez les résultats et comparez-les avec les modèles individuels.

    Pasting avec Arbres de Décision :
        Utilisez un modèle de Pasting avec un arbre de décision comme modèle de base.
        Utilisez le critère d'entropie pour mesurer la pureté des nœuds dans les arbres.
        Ajustez le modèle sur l'ensemble d'entraînement en utilisant 50 estimateurs.
        Évaluez la performance sur l'ensemble de test en utilisant les métriques suivantes :
            Score F1
            Score R2
        Affichez les résultats et comparez-les avec les modèles individuels.

    Questions de Discussion :
        Comparez les résultats obtenus avec le Bagging et le Pasting. Quelles observations pouvez-vous faire sur la performance des deux méthodes ?
        Expliquez comment l'utilisation de l'échantillonnage avec remplacement (Bagging) peut influencer la performance par rapport à l'utilisation de l'échantillonnage sans remplacement (Pasting).
        Comment les métriques F1 et R2 sont-elles utilisées pour évaluer la performance d'un modèle de régression ? Quelles informations fournissent-elles ?

N'oubliez pas d'utiliser les hyperparamètres vus précédemment (nombre d'estimateurs, critère de mesure de la pureté, etc.) pour ajuster vos modèles. Bon exercice !

Chargement des données :
        Chargez le jeu de données depuis le fichier wine.csv.
        Supprimez les lignes avec des valeurs manquantes dans la colonne price.

In [4]:
# Chargement du dataset
url = "datasets/wine.csv" 
data = pd.read_csv(url)

# 1. Nettoyage des données
# Remplir les valeurs manquantes (par la moyenne pour les variables numériques, par exemple)
# Convertir les variables catégorielles en variables numériques
data['price'].fillna(data['price'].mean(), inplace=True)  # Remplacer les valeurs manquantes dans la colonne 'price' par la moyenne

# Remplir les valeurs manquantes dans d'autres colonnes numériques si nécessaire
# Exemple : data['col_numerique'].fillna(data['col_numerique'].mean(), inplace=True)

# Convertir les variables catégorielles en variables numériques
data_encoded = pd.get_dummies(data, columns=['country', 'province', 'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title', 'variety', 'winery'])

# Séparation des features (X) et de la target variable (y) pour la régression
X_reg = data_encoded.drop(['price'], axis=1)  # Features pour la régression
y_reg = data_encoded['price']  # Target variable pour la régression

# Divisez le dataset en ensembles d'entraînement et de test
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Entraînement du modèle de Random Forest pour la régression
reg_model = RandomForestRegressor(n_estimators=100, random_state=42)
reg_model.fit(X_reg_train, y_reg_train)

# Prédictions sur l'ensemble de test
y_reg_pred = reg_model.predict(X_reg_test)

# Évaluation des performances pour la régression
mse = mean_squared_error(y_reg_test, y_reg_pred)
rmse = mse**0.5
print(f"RMSE (Régression) : {rmse}")

# 3. Random Forest pour la Classification (Variété de Vin)
# Convertissez la variable de variété en une variable numérique si nécessaire
# (Assurez-vous que les valeurs de la variété sont numériques ou encodez-les)

# Séparation des features (X) et de la target variable (y)
X_class = data.drop(['price', 'variety'], axis=1)  # Features pour la classification
y_class = data['variety']  # Target variable pour la classification

# Divisez le dataset en ensembles d'entraînement et de test
X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Entraînement du modèle de Random Forest pour la classification
class_model = RandomForestClassifier(n_estimators=100, random_state=42)
class_model.fit(X_class_train, y_class_train)

# Prédictions sur l'ensemble de test
y_class_pred = class_model.predict(X_class_test)

# Évaluation des performances pour la classification
accuracy = accuracy_score(y_class_test, y_class_pred)
print(f"Accuracy (Classification) : {accuracy}")

# 4. Bagging ou Pasting
# Comparez les performances du Random Forest avec bagging et du Random Forest avec pasting en ajustant les hyperparamètres
# (Changer le nombre d'estimateurs, observer les résultats)

# 5. Analyse des Résultats
# Analysez et interprétez les résultats obtenus, identifiez les hyperparamètres optimaux pour chaque modèle



: 

Préparation des données :
        Séparez les variables indépendantes (X) et la variable cible (y).
        Divisez l'ensemble de données en ensembles d'entraînement et de test.

Bagging avec Arbres de Décision :
        Utilisez un modèle de Bagging avec un arbre de décision comme modèle de base.
        Utilisez le critère de Gini pour mesurer la pureté des nœuds dans les arbres.
        Ajustez le modèle sur l'ensemble d'entraînement en utilisant 50 estimateurs.
        Évaluez la performance sur l'ensemble de test en utilisant les métriques suivantes :
            Score F1
            Score R2
        Affichez les résultats et comparez-les avec les modèles individuels.

Pasting avec Arbres de Décision :
        Utilisez un modèle de Pasting avec un arbre de décision comme modèle de base.
        Utilisez le critère d'entropie pour mesurer la pureté des nœuds dans les arbres.
        Ajustez le modèle sur l'ensemble d'entraînement en utilisant 50 estimateurs.
        Évaluez la performance sur l'ensemble de test en utilisant les métriques suivantes :
            Score F1
            Score R2
        Affichez les résultats et comparez-les avec les modèles individuels.

Questions de Discussion :
        Comparez les résultats obtenus avec le Bagging et le Pasting. Quelles observations pouvez-vous faire sur la performance des deux méthodes ?
        Expliquez comment l'utilisation de l'échantillonnage avec remplacement (Bagging) peut influencer la performance par rapport à l'utilisation de l'échantillonnage sans remplacement (Pasting).
        Comment les métriques F1 et R2 sont-elles utilisées pour évaluer la performance d'un modèle de régression ? Quelles informations fournissent-elles ?

N'oubliez pas d'utiliser les hyperparamètres vus précédemment (nombre d'estimateurs, critère de mesure de la pureté, etc.) pour ajuster vos modèles. Bon exercice !

QCM sur le Bagging, le Pasting et l'Ensemble d'Arbres de Décision :

    Quelle est la principale différence entre le Bagging et le Pasting ?
        a. La fonction de coût utilisée
        b. L'échantillonnage avec ou sans remplacement
        c. Le nombre d'estimateurs dans l'ensemble
        d. La profondeur des arbres de décision

    Réponse : b. L'échantillonnage avec ou sans remplacement

    Quel est le rôle de l'échantillonnage bootstrap dans le Bagging ?
        a. Échantillonner sans remplacement pour créer des sous-ensembles d'entraînement
        b. Introduire de la diversité entre les modèles
        c. Maximiser la corrélation entre les modèles
        d. Minimiser la variance des modèles

    Réponse : b. Introduire de la diversité entre les modèles

    Pourquoi le Bagging peut-il être plus robuste que l'utilisation d'un seul modèle de base ?
        a. Il utilise des modèles de base plus complexes
        b. Il introduit de la diversité entre les modèles
        c. Il utilise une fonction de coût différente
        d. Il ajuste davantage les modèles aux données d'entraînement

    Réponse : b. Il introduit de la diversité entre les modèles

    Quels sont les hyperparamètres communs pour ajuster un modèle Bagging avec des arbres de décision ?
        a. learning_rate et n_estimators
        b. max_depth et min_samples_split
        c. n_estimators et max_features
        d. C et kernel

    Réponse : c. n_estimators et max_features

    Quel critère est utilisé pour mesurer la pureté d'un nœud lors de la construction d'un arbre de décision dans le Bagging ?
        a. Variance
        b. Gini Index
        c. Entropie
        d. MSE (Mean Squared Error)

    Réponse : b. Gini Index

    Quel est l'objectif principal du Bagging en termes de performance du modèle ?
        a. Maximiser la variance
        b. Minimiser la diversité
        c. Minimiser la corrélation entre les modèles
        d. Minimiser la précision

    Réponse : c. Minimiser la corrélation entre les modèles

    Quelle métrique est souvent utilisée pour évaluer la performance d'un modèle Bagging sur des données de test ?
        a. Aire sous la courbe ROC (AUC-ROC)
        b. Précision (Precision)
        c. Exactitude (Accuracy)
        d. Score F1

    Réponse : c. Exactitude (Accuracy)